In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from scipy.stats import uniform, randint

#import xgboost as xgb
import lightgbm as lgb


In [71]:
def diagnostic_stats(ytrue, ypred):
    """
    https://stats.stackexchange.com/questions/142248/difference-between-r-square-and-rmse-in-linear-regression
    
    https://www.sciencedirect.com/topics/engineering/mean-bias-error
    """
    n = len(ytrue)

    # Check that the ytrue and ypred are equal length vector.
    assert n == len(ypred)
    
    # sum squared error
    sse = np.sum((ytrue - ypred)**2)
    
    # root mean square error
    rmse = np.sqrt(sse/n)

    # total sum of squares
    tss = np.sum((ytrue - np.mean(ytrue))**2)
    tst = np.sum((ypred - np.mean(ypred))**2)
    tstp = tst**0.5
    tssp = tss**0.5
    
    soorat = np.sum((ytrue-np.mean(ytrue))*(ypred-np.mean(ypred)))
    
    # Rsquare
    ##rsqr = 1 - sse/tss
    rsqr = (soorat/(tssp*tstp))**2

    # Mean biased error
    mbe = np.mean(ytrue - ypred)
    
    
    # IOAD
    num = np.sum((ytrue - ypred)**2)
    denom = np.abs(ytrue - ypred) + np.abs(ytrue + ypred)
    ioad = 1 - num/np.sum(denom**2)

    print("RMSE: %1.3f, R^2: %1.3f, MBE: %1.3f, IOAD: %1.3f"%(rmse, rsqr, mbe, ioad))
    
    return rmse, rsqr, mbe, ioad

In [72]:
evidf = pd.read_csv('../data_out/Gingin_EVI_processed.csv', parse_dates=['DateTime'], index_col='DateTime')
tempdf = pd.read_csv('rnn_data_prajwal.csv', parse_dates=['DateTime'], index_col='DateTime')
tempdf.drop('Unnamed: 0', axis=1, inplace=True)

In [73]:
df = pd.merge(tempdf, evidf, how='left', on='DateTime')

In [74]:
df.index.min(), df.index.max()

(Timestamp('2013-01-01 01:00:00'), Timestamp('2014-01-01 00:00:00'))

In [75]:
Xvar = ['Ta', 'Ws', 'Fg', 'VPD', 'Fn', 'q', 'Ts', 'Sws', 'EVI']
yvar = 'Fc'

In [76]:
df.head()

,Fc,Ta,Ws,Fg,VPD,Fn,q,Ts,Sws,EVI
DateTime,,,,,,,,,,
2013-01-01 01:00:00,3.070399,26.998711,2.08,-20.984654,1.183521,-39.635899,0.014801,32.291790,0.032662,0.225559
2013-01-01 01:30:00,2.948313,26.629910,2.47,-19.131921,1.067606,-35.979652,0.015045,32.036301,0.032636,0.225565
2013-01-01 02:00:00,2.369790,26.294310,1.95,-18.278720,1.002275,-38.033844,0.015030,31.798710,0.032619,0.225570
2013-01-01 02:30:00,0.105580,25.739479,1.55,-17.862080,0.927464,-41.878986,0.014810,31.572710,0.032582,0.225576
2013-01-01 03:30:00,2.085828,24.548740,2.69,-21.647482,0.792163,-57.483334,0.014242,31.069750,0.032534,0.225588


### Train-test split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df[Xvar], df[yvar], 
                                                    test_size=0.20, random_state=40, shuffle=True)

#### Data Day-night split

In [78]:
X_train_night = X_train.between_time('18:00', '07:00')
X_test_night = X_test.between_time('18:00', '07:00')
y_train_night = y_train.between_time('18:00', '07:00')
y_test_night = y_test.between_time('18:00', '07:00')

In [79]:
X_train_day = X_train.between_time('07:00', '18:00')
X_test_day = X_test.between_time('07:00', '18:00')
y_train_day = y_train.between_time('07:00', '18:00')
y_test_day = y_test.between_time('07:00', '18:00')

#### Scaling data

In [80]:
def scaling(X, y):
    scaler_test = StandardScaler()
    X_scaled = scaler_test.fit_transform(X)
    y_mean, y_std = y.mean(), y.std()
    y_scaled = (y - y_mean)/y_std
    return X_scaled, y_scaled, y_mean, y_std

In [81]:
X_test_scaled, y_test_scaled, y_test_mean, y_test_std = scaling(X_test, y_test)
X_train_scaled, y_train_scaled, y_train_mean, y_train_std = scaling(X_train, y_train)

X_test_day_scaled, y_test_day_scaled, y_test_day_mean, y_test_day_std = scaling(X_test_day, y_test_day)
X_test_night_scaled, y_test_night_scaled, y_test_night_mean, y_test_night_std = scaling(X_test_night, 
                                                                                        y_test_night)

X_train_day_scaled, y_train_day_scaled, y_train_day_mean, y_train_day_std = scaling(X_train_day, y_train_day)
X_train_night_scaled, y_train_night_scaled, y_train_night_mean, y_train_night_std = scaling(X_train_night, 
                                                                                            y_train_night)

In [82]:
print('Single', X_train.shape)
print('Night', X_train_night.shape)
print('Day', X_train_day.shape)


Single (7212, 9)
Night (2293, 9)
Day (5202, 9)


### A. Train single model

In [83]:
def rmsle_cv(model, X_train, y_train, n_folds):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    score = cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf)
    rmse= np.sqrt(-score)
    return(rmse), model.fit(X_train, y_train)

def rmsle_randomsearchcv(model, params, X_train, y_train, n_folds, random_state=0):
    clf = RandomizedSearchCV(model, params, random_state=random_state, cv=n_folds, return_train_score=True, 
                         scoring="neg_mean_squared_error")
    search = clf.fit(X_train, y_train)
    rmse= np.sqrt(-search.best_score_)
    return rmse, search

In [84]:
N_FOLDS = 3

# Hyperparameter Optz
params = dict(num_leaves = randint(2,10), 
              n_estimators = randint(100,1000),
              learning_rate = uniform(1e-3, 1),
              min_data_in_leaf = randint(2,10),
              objective=['regression'])

cls_ = lgb.LGBMRegressor()

score, cls_ = rmsle_randomsearchcv(cls_, params, X_train_scaled, y_train_scaled, N_FOLDS)
print("\nSingle LGB score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


Single LGB score: 0.6762 (+/-0.0000)



In [85]:
model_single = lgb.LGBMRegressor(**cls_.best_params_)

score, model_single = rmsle_cv(model_single, X_train_scaled, y_train_scaled, N_FOLDS)
print("\nSingle LGB score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


Single LGB score: 0.6761 (+/-0.0036)



### B. Train day-night model

In [86]:
# Hyperparameter Optz
params = dict(num_leaves = randint(2,10), 
              n_estimators = randint(100,1000),
              learning_rate = uniform(1e-3, 1),
              min_data_in_leaf = randint(2,10),
              objective=['regression'])

cls_day = lgb.LGBMRegressor()

score, cls_day = rmsle_randomsearchcv(cls_day, params, X_train_day_scaled, y_train_day_scaled, N_FOLDS)
print("\nDay LGBM score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


Day LGBM score: 0.7726 (+/-0.0000)



In [87]:
model_day = lgb.LGBMRegressor(**cls_day.best_params_)

score, model_day = rmsle_cv(model_day, X_train_day_scaled, y_train_day_scaled, N_FOLDS)
print("\nDay LGBM score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


Day LGBM score: 0.7725 (+/-0.0126)



In [88]:
# Hyperparameter Optz
params = dict(num_leaves = randint(2,10), 
              n_estimators = randint(100,1000),
              learning_rate = uniform(1e-3, 1),
              min_data_in_leaf = randint(2,10),
              objective=['regression'])

cls_night = lgb.LGBMRegressor()

score, cls_night = rmsle_randomsearchcv(cls_night, params, X_train_night_scaled, y_train_night_scaled, N_FOLDS)
print("\nNightGBM score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


NightGBM score: 0.9297 (+/-0.0000)



In [89]:
model_night = lgb.LGBMRegressor(**cls_night.best_params_)

score, model_night = rmsle_cv(model_night, X_train_night_scaled, y_train_night_scaled, N_FOLDS)
print("\nNight LGBM score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))


Night LGBM score: 0.9282 (+/-0.0539)



### Score comparison

In [90]:
y_test_pred = model_single.predict(X_test_scaled)
y_test_night_pred = model_night.predict(X_test_night_scaled)
y_test_day_pred = model_day.predict(X_test_day_scaled)

In [93]:
print('Night LGBM')
diagnostic_stats(y_test_night_pred, y_test_night_pred*y_test_night_std + y_test_night_mean);
print('Day LGBM')
diagnostic_stats(y_test_day_pred, y_test_day_pred*y_test_day_std + y_test_day_mean);

Night LGBM
RMSE: 2.067, R^2: 1.000, MBE: -1.996, IOAD: 0.783
Day LGBM
RMSE: 4.169, R^2: 1.000, MBE: 3.643, IOAD: 0.787


In [96]:
print('Single LGBM')
diagnostic_stats(y_test, y_test_pred*y_test_std + y_test_mean);
print('Day/Night Merged LGBM')
diagnostic_stats(np.concatenate((y_test_day_pred, y_test_night_pred)), 
                 np.concatenate((y_test_day_pred*y_test_day_std + y_test_day_mean,
                                 y_test_night_pred*y_test_night_std + y_test_night_mean)));

Single LGBM
RMSE: 2.957, R^2: 0.579, MBE: 0.023, IOAD: 0.922
Day/Night Merged LGBM
RMSE: 3.686, R^2: 0.423, MBE: 2.010, IOAD: 0.787
